<a href="https://colab.research.google.com/github/ricospeloacaso/python_para_investimentos/blob/master/16_CVM_Os_Melhores_e_os_Piores_Fundos_de_Investimento_do_mes_Python_para_Investimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Ricos pelo Acaso

*   Link para o vídeo: https://youtu.be/NHCUUZOvk7k
---
*   Base de Dados: http://dados.cvm.gov.br/



###Coletando os dados da CVM

In [0]:
import pandas as pd
pd.set_option("display.max_colwidth", 150)
#pd.options.display.float_format = '{:.2f}'.format

>Funções que buscam dados no site da CVM e retornam um DataFrame Pandas:


In [0]:
def busca_informes_cvm(ano, mes):
  url = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{:02d}{:02d}.csv'.format(ano,mes)
  return pd.read_csv(url, sep=';')

In [0]:
def busca_cadastro_cvm(ano, mes, dia):
  url = 'http://dados.cvm.gov.br/dados/FI/CAD/DADOS/inf_cadastral_fi_{}{:02d}{:02d}.csv'.format(ano, mes, dia)
  return pd.read_csv(url, sep=';', encoding='ISO-8859-1')

>Buscando dados no site da CVM

In [0]:
informes_diarios = busca_informes_cvm(2020,4)

In [0]:
informes_diarios

,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,00.017.024/0001-53,2020-04-01,1127984.12,27.380981,1121661.69,0.0,0.0,1
1,00.017.024/0001-53,2020-04-02,1128125.49,27.383353,1121758.88,0.0,0.0,1
2,00.017.024/0001-53,2020-04-03,1128288.77,27.385179,1121833.68,0.0,0.0,1
3,00.017.024/0001-53,2020-04-06,1128434.93,27.387583,1121932.15,0.0,0.0,1
4,00.017.024/0001-53,2020-04-07,1128235.55,27.389569,1122013.50,0.0,0.0,1
...,...,...,...,...,...,...,...,...
345798,97.929.213/0001-34,2020-04-23,62837349.30,8.267813,62829766.78,0.0,0.0,2
345799,97.929.213/0001-34,2020-04-24,62585452.40,8.234617,62577500.24,0.0,0.0,2
345800,97.929.213/0001-34,2020-04-27,62980521.79,8.286556,62972198.43,0.0,0.0,2
345801,97.929.213/0001-34,2020-04-28,63000881.62,8.289186,62992186.98,0.0,0.0,2


In [0]:
cadastro_cvm = busca_cadastro_cvm(2020,5,1)

In [0]:
cadastro_cvm

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,DT_INI_EXERC,DT_FIM_EXERC,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,TRIB_LPRAZO,INVEST_QUALIF,TAXA_PERFM,INF_TAXA_PERFM,TAXA_ADM,INF_TAXA_ADM,VL_PATRIM_LIQ,DT_PATRIM_LIQ,DIRETOR,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
0,36.498.416/0001-29,03 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,2020-04-09,2020-04-09,NaN,EM FUNCIONAMENTO NORMAL,2020-04-27,2020-04-27,2020-04-27,2020-06-30,Fundo Multimercado,2020-04-09,NaN,Aberto,S,N,S,N,NaN,NaN,NaN,NaN,1.500212e+08,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
1,36.729.776/0001-94,03 MASTER FUNDO DE INVESTIMENTO EM ACOES,2020-04-08,2020-04-08,NaN,FASE PRÉ-OPERACIONAL,2020-04-08,NaN,NaN,NaN,Fundo de Ações,2020-04-08,NaN,Aberto,N,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
2,36.499.643/0001-79,03 MASTER FUNDO DE INVESTIMENTO MULTIMERCADO,2020-04-09,2020-04-09,NaN,EM FUNCIONAMENTO NORMAL,2020-04-27,2020-04-27,2020-04-27,2020-06-30,Fundo Multimercado,2020-04-09,NaN,Aberto,N,N,S,N,NaN,NaN,NaN,NaN,1.498389e+08,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
3,36.016.411/0001-12,051 ALLOCATION FUNDO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CREDITO PRIVADO,2020-04-03,2020-04-02,NaN,FASE PRÉ-OPERACIONAL,2020-04-02,NaN,NaN,NaN,Fundo Multimercado,2020-04-02,NaN,Aberto,S,N,S,S,NaN,NaN,NaN,NaN,NaN,NaN,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
4,33.913.620/0001-70,051 BRUCUTU FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO EXTERIOR,2019-07-19,2019-06-04,NaN,EM FUNCIONAMENTO NORMAL,2019-08-05,2019-08-05,2019-08-05,2020-05-31,Fundo Multimercado,2019-06-04,DI de um dia,Fechado,N,S,NaN,S,20.0,NaN,0.2,NaN,2.442650e+07,2020-04-29,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36770,31.340.900/0001-01,ZURICH VALORA PREVIDENCIARIO FI EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA CREDITO PRIVADO,2018-12-19,2018-12-19,NaN,EM FUNCIONAMENTO NORMAL,2019-03-27,2019-03-27,2020-01-01,2020-12-31,Fundo de Renda Fixa,2018-12-19,NaN,Aberto,S,S,N,S,20.0,O fundo cobrara 20.00 % do que exceder 100.00 % o indice CDIE. A taxa de performance sera paga no(s) mes(es) de Janeiro e Julho,1.1,NaN,2.084469e+07,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,07.559.989/0001-17,VALORA GESTÃO DE INVESTIMENTOS LTDA.,61

###Manipulando os dados da CVM

>Definindo filtros para os Fundos de Investimento


In [0]:
minimo_cotistas = 100

>Manipulando os dados e aplicando filtros

In [0]:
fundos = informes_diarios[informes_diarios['NR_COTST'] >= minimo_cotistas].pivot(index='DT_COMPTC', columns='CNPJ_FUNDO', values=['VL_TOTAL',	'VL_QUOTA',	'VL_PATRIM_LIQ',	'CAPTC_DIA',	'RESG_DIA'])

In [0]:
fundos

VL_TOTAL  ...           RESG_DIA
CNPJ_FUNDO 00.068.305/0001-35  ... 97.519.794/0001-36
DT_COMPTC                      ...                   
2020-04-01        52347266.29  ...          971670.38
2020-04-02        52286813.89  ...          434254.39
2020-04-03        52281604.34  ...         1015272.40
2020-04-06        52211795.07  ...          735636.46
2020-04-07        52099859.60  ...          182700.32
2020-04-08        52013743.91  ...         2535908.36
2020-04-09        52007899.94  ...         1215339.58
2020-04-13        51833455.31  ...         2234235.17
2020-04-14        51858224.53  ...         1117338.09
2020-04-15        51857507.07  ...         1004980.44
2020-04-16        51832604.49  ...          368631.57
2020-04-17        51810701.61  ...          582196.31
2020-04-20        51820626.96  ...          824703.47
2020-04-22        51794821.72  ...          327857.26
2020-04-23        51776604.59  ...          641327.73
2020-04-24        51908821.85  ...         1642396.26
2020-04-27        51840479.65  ...          758294.36
2020-04-28        51899720.19  ...          612883.79
2020-04-29        51637623.48  ...         1217948.08
2020-04-30        51697882.22  ...          914635.21

[20 rows x 13430 columns]

>Normalizando os dados de cotas para efeitos comparativos

In [0]:
cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]

In [0]:
cotas_normalizadas

###Fundos de Investimento com os melhores desempenhos em Abril de 2020

In [0]:
melhores = pd.DataFrame()
melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:5] - 1) * 100
melhores

,retorno(%)
CNPJ_FUNDO,
34.218.752/0001-45,44.941707
13.476.201/0001-01,42.572977
18.832.847/0001-06,39.202412
35.625.840/0001-24,38.137070
35.650.540/0001-03,37.972896


>Buscando dados dos Fundos de Investimento pelo CNPJ

In [0]:
for cnpj in melhores.index:
  fundo = cadastro_cvm[cadastro_cvm['CNPJ_FUNDO'] == cnpj]
  melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
  melhores.at[cnpj, 'Classe'] = fundo['CLASSE'].values[0]
  melhores.at[cnpj, 'PL'] = fundo['VL_PATRIM_LIQ'].values[0]

In [0]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
34.218.752/0001-45,44.941707,VITREO EXPONENCIAL FUNDO DE INVESTIMENTO EM AÇÕES INVESTIMENTO NO EXTERIOR,Fundo de Ações,1.346177e+08
13.476.201/0001-01,42.572977,FATOR SINERGIA FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,1.055573e+08
18.832.847/0001-06,39.202412,VERSA LONG BIASED FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,1.477850e+08
35.625.840/0001-24,38.137070,TREND XP INC FI EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES INVESTIMENTO NO EXTERIOR,Fundo de Ações,1.812153e+08
35.650.540/0001-03,37.972896,TREND XP INC DOMINUS FI EM COTAS DE FI EM AÇÕES INVESTIMENTO NO EXTERIOR,Fundo de Ações,1.856156e+07


###Fundos de Investimento com os piores desempenhos em Abril de 2020

In [0]:
piores = pd.DataFrame()
piores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=True)[:5] - 1) * 100
piores

,retorno(%)
CNPJ_FUNDO,
35.780.106/0001-30,-17.063642
31.188.723/0001-90,-9.569397
16.599.959/0001-25,-5.789261
10.869.628/0001-81,-5.664139
10.590.125/0001-72,-5.214088


>Buscando dados dos Fundos de Investimento pelo CNPJ

In [0]:
for cnpj in piores.index:
  fundo = cadastro_cvm[cadastro_cvm['CNPJ_FUNDO'] == cnpj]
  piores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
  piores.at[cnpj, 'Classe'] = fundo['CLASSE'].values[0]
  piores.at[cnpj, 'PL'] = fundo['VL_PATRIM_LIQ'].values[0]

In [0]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
35.780.106/0001-30,-17.063642,JOURNEY CAPITAL VITREO RDVT11 FUNDO INCENTIVADO DE INVESTIMENTO EM INFRAESTRUTURA RENDA FIXA,Fundo de Renda Fixa,8578774.80
31.188.723/0001-90,-9.569397,ULTRA PERFORMANCE CTM FUNDO DE INVESTIMENTO EM ACOES - BDR NIVEL 1,Fundo de Ações,36855942.92
16.599.959/0001-25,-5.789261,TAGUS TOP FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO II,Fundo de Renda Fixa,74482136.46
10.869.628/0001-81,-5.664139,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,20684030.83
10.590.125/0001-72,-5.214088,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,6563087.43
